In [ ]:
!pip install -U transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
!pip install transformers datasets


In [ ]:
from google.colab import files
uploaded = files.upload()  # ordered_curriculum.txt 업로드


Saving small_ordered.txt to small_ordered.txt


In [ ]:
from datasets import Dataset
import random

with open("small_ordered.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

random.shuffle(lines)
split1 = int(0.9 * len(lines))
split2 = int(0.95 * len(lines))
train_lines = lines[:split1]
val_lines = lines[split1:split2]
test_lines = lines[split2:]

train_dataset = Dataset.from_dict({"text": train_lines})
val_dataset = Dataset.from_dict({"text": val_lines})
test_dataset = Dataset.from_dict({"text": test_lines})


In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", use_auth_token=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", use_auth_token=False)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1932: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4056: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    encoding["labels"] = encoding["input_ids"]
    return encoding

tokenized_train = train_dataset.map(tokenize_function, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, remove_columns=["text"])


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results-curriculum",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)



In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,0.886700
200,0.083500
300,0.074800
400,0.069900
500,0.071000
600,0.066900
700,0.067000
800,0.064400
900,0.063900
1000,0.062100


TrainOutput(global_step=3375, training_loss=0.08345967589484321, metrics={'train_runtime': 419.091, 'train_samples_per_second': 64.425, 'train_steps_per_second': 8.053, 'total_flos': 1763721216000000.0, 'train_loss': 0.08345967589484321, 'epoch': 3.0})

In [ ]:
trainer.save_model("checkpoint-curriculum")


In [ ]:
import math
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


>>> Perplexity: 1.05


In [ ]:
!zip -r checkpoint-curriculum.zip checkpoint-curriculum
from google.colab import files
files.download("checkpoint-curriculum.zip")


  adding: checkpoint-curriculum/ (stored 0%)
  adding: checkpoint-curriculum/training_args.bin (deflated 52%)
  adding: checkpoint-curriculum/config.json (deflated 52%)
  adding: checkpoint-curriculum/model.safetensors (deflated 7%)
  adding: checkpoint-curriculum/generation_config.json (deflated 24%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>